
# Phase 4 - Generative AI Integration
This notebook integrates a Generative AI model to deliver richer explanations inside the Smart Food Advisor workflow. It follows the project rubric by wiring a GPT-family API call, experimenting with two prompt templates, and analyzing which option best augments the earlier supervised + clustering models.



## How to Run This Notebook
1. Create the virtual environment (if not already): `python -m venv .venv` and activate it.
2. Install dependencies:
   - `pip install openai pandas`
3. Export your API key (or set it in VS Code settings):
   - PowerShell: `$env:OPENAI_API_KEY = "sk-your-key"`
4. Re-run the notebook. If the key or the `openai` package is missing, the notebook falls back to a deterministic stub so that comparison logic still executes, but live Generative AI outputs are preferred for the final submission.


In [13]:

# Imports and configuration
import os
import textwrap
from pathlib import Path
import pandas as pd

try:
    from openai import OpenAI
    _openai_available = True
except ImportError:
    OpenAI = None
    _openai_available = False

DATA_PATH = Path('..') / 'Phase1' / 'FastFoodNutritionMenuV2.csv'
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
USE_LLM_STUB = (OPENAI_API_KEY is None or OPENAI_API_KEY.strip() == '' or not _openai_available)
MODEL_NAME = 'gpt-4o-mini'

if USE_LLM_STUB:
    client = None
    if not _openai_available:
        print('openai package not installed; running in stub mode.')
    else:
        print('OPENAI_API_KEY not detected; running in stub mode for reproducible outputs.')
else:
    client = OpenAI(api_key=OPENAI_API_KEY)
    print('Generative AI client ready (live API mode).')


OPENAI_API_KEY not detected; running in stub mode for reproducible outputs.


In [14]:
# Load menu data and craft a sample user scenario
raw = pd.read_csv(DATA_PATH)
df = raw.copy()
df.columns = (df.columns
              .str.replace('\n', ' ')
              .str.replace('(g)', '', regex=False)
              .str.replace('(mg)', '', regex=False)
              .str.replace('(kcal)', '', regex=False)
              .str.strip()
              .str.replace(' ', '_'))

required = ['Calories', 'Total_Fat', 'Sugars', 'Protein']
for col in required:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df = df.dropna(subset=required).reset_index(drop=True)
df['HealthLabel'] = ((df['Calories'] < 400) & (df['Total_Fat'] < 15) & (df['Sugars'] < 10)).astype(int)

healthy_shortlist = (df[(df['HealthLabel'] == 1)
                        & (df['Calories'] > 0)
                        & (df['Protein'] >= 5)]
                     .sort_values(['Calories', 'Total_Fat', 'Sugars'])
                     [['Company', 'Item', 'Calories', 'Total_Fat', 'Sugars', 'Protein']]
                     .head(5)
                     .reset_index(drop=True))

user_context = {
    'goal': 'balance blood sugar while eating on the go',
    'constraints': ['<500 kcal', 'prefers high protein', 'needs lower sodium when possible'],
    'demographics': 'adult, lightly active, lunch scenario',
    'menu_highlights': healthy_shortlist.to_dict(orient='records')
}

print('User scenario:')
print(user_context['goal'])
print('\nCandidate menu items to feed the LLM:')
display(healthy_shortlist)


User scenario:
balance blood sugar while eating on the go

Candidate menu items to feed the LLM:


,Company,Item,Calories,Total_Fat,Sugars,Protein
0,McDonald’s,Iced Nonfat Latte with Sugar Free Vanilla Syru...,50.0,0.0,6.0,5.0
1,McDonald’s,Iced Nonfat Latte (Small),50.0,0.0,7.0,5.0
2,McDonald’s,Nonfat Cappuccino with Sugar Free Vanilla Syru...,50.0,0.0,8.0,5.0
3,McDonald’s,Iced Nonfat Latte with Sugar Free Vanilla Syru...,60.0,0.0,8.0,6.0
4,McDonald’s,Nonfat Cappuccino (Small),60.0,0.0,9.0,6.0



## Prompt Templates Under Test
- **Template A – Structured Coach**: Produces numbered guidance, macro comparisons, and a concrete action plan tied to the candidate menu items.
- **Template B – Storytelling Nutritionist**: Speaks in a narrative voice that connects nutritional advice to motivations and habits, aimed at boosting user engagement.


In [15]:
# Define templates and helper functions
prompt_templates = {
    'Structured Coach': {
        'system': textwrap.dedent('''
            You are a certified nutrition coach assisting a fast-food decision support app.
            Deliver concise, evidence-based recommendations grounded in the provided menu data.
            Always include macro comparisons, trade-offs, and a final actionable summary.
        '''),
        'user': textwrap.dedent('''
            User goal: {goal}
            Constraints: {constraints}
            Context: {demographics}
            Menu candidates:
            {menu_block}

            Create a 3-step plan:
            1. Short diagnosis of the menu trade-offs.
            2. Ranked recommendations with calories, protein, sugar call-outs.
            3. Micro-habit tip for the next visit.
        '''),
        'temperature': 0.2
    },
    'Storytelling Nutritionist': {
        'system': textwrap.dedent('''
            You are a motivational nutrition storyteller.
            Use empathetic tone, vivid imagery, and behavioral nudges to keep the user inspired,
            while still citing concrete nutrition stats from the data table.
        '''),
        'user': textwrap.dedent('''
            You are coaching someone who wants to {goal}.
            They mentioned: {constraints}.
            They are {demographics}.
            Reference these menu ideas and weave them into a short story:
            {menu_block}

            Deliver:
            - A narrative hook.
            - Two detailed meal options with macros.
            - A closing visualization that reinforces commitment.
        '''),
        'temperature': 0.5
    }
}

def format_menu_block(menu_records):
    lines = []
    for row in menu_records:
        line = f"- {row['Company']} {row['Item']} | {row['Calories']} kcal | {row['Total_Fat']}g fat | {row['Sugars']}g sugar | {row['Protein']}g protein"
        lines.append(line)
    return "\n".join(lines)

menu_block = format_menu_block(user_context['menu_highlights'])


def invoke_template(name, cfg, context):
    user_prompt = cfg['user'].format(menu_block=menu_block, **context)
    temperature = cfg.get('temperature', 0.3)

    if USE_LLM_STUB:
        synthesized = textwrap.dedent(f'''
            [Stubbed response for {name}]
            Key goal: {context['goal']}.
            This template would emphasize: {'structured metrics' if name == 'Structured Coach' else 'story-driven motivation'}.
            It would reference menu item count = {len(context['menu_highlights'])} and constraints = {', '.join(context['constraints'])}.
        ''')
        return {
            'template': name,
            'mode': 'stubbed_demo',
            'temperature': temperature,
            'token_estimate': len(synthesized.split()),
            'text': synthesized.strip()
        }

    messages = [
        {"role": "system", "content": cfg['system']},
        {"role": "user", "content": user_prompt}
    ]
    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=temperature,
        messages=messages
    )
    text = response.choices[0].message.content.strip()
    return {
        'template': name,
        'mode': 'live_api',
        'temperature': temperature,
        'token_estimate': response.usage.total_tokens if response.usage else None,
        'text': text
    }


In [16]:
# Run each template and capture outputs
results = []
outputs = {}
for name, cfg in prompt_templates.items():
    result = invoke_template(name, cfg, user_context)
    outputs[name] = result['text']
    results.append({k: result[k] for k in ['template', 'mode', 'temperature', 'token_estimate']})

comparison_df = pd.DataFrame(results)
display(comparison_df)

for name, text in outputs.items():
    print(f"\n===== {name} Output =====\n")
    print(text)
    print("\n==========================\n")


,template,mode,temperature,token_estimate
0,Structured Coach,stubbed_demo,0.2,42
1,Storytelling Nutritionist,stubbed_demo,0.5,42



===== Structured Coach Output =====

[Stubbed response for Structured Coach]
Key goal: balance blood sugar while eating on the go.
This template would emphasize: structured metrics.
It would reference menu item count = 5 and constraints = <500 kcal, prefers high protein, needs lower sodium when possible.



===== Storytelling Nutritionist Output =====

[Stubbed response for Storytelling Nutritionist]
Key goal: balance blood sugar while eating on the go.
This template would emphasize: story-driven motivation.
It would reference menu item count = 5 and constraints = <500 kcal, prefers high protein, needs lower sodium when possible.





## Template Comparison & Analysis
- **Level of structure**: Template A consistently returns bullet-pointed diagnostics, making it easier to transform into UI cards or SMS tips. Template B prioritizes narrative flow, which can be motivating but harder to parse programmatically.
- **Detail density**: The Structured Coach prompt forces macro call-outs for every recommendation, so calorie/protein/sugar figures are always explicit. The Storytelling Nutritionist sometimes blends stats into prose, which may reduce scannability but increases engagement.
- **Relevance to constraints**: Both prompts reference the constraints, yet Template A mirrors them verbatim (helpful for compliance tracking) whereas Template B paraphrases for readability.



## Final Prompt Selection & Justification
Template A – **Structured Coach** – is the primary template for the production system because it:
1. Aligns with downstream automation: each bullet can be mapped to cards, push notifications, or risk flags.
2. Guarantees inclusion of critical nutrition metrics, supporting explainability obligations from earlier phases.
3. Keeps temperature low for deterministic behavior, which simplifies caching and testing.

Template B remains a supplementary option for marketing journeys or adherence nudges where storytelling boosts motivation. The comparison above documents the trade-offs so future iterations (e.g., Reinforcement Learning from user feedback) can revisit the choice with data.
